In [1]:
import climate_learn as cl
from climate_learn.data import DataModule
from climate_learn.data.climate_dataset.args import ERA5Args
from climate_learn.data.dataset.args import ShardDatasetArgs, MapDatasetArgs
from climate_learn.data.task.args import ForecastingArgs

import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy("file_system")

In [2]:
root = "/home/data/datasets/weatherbench/era5/5.625deg"
dataset = "era5"
variables = [
    "2m_temperature",
    # "geopotential",
    # "temperature",
    # "specific_humidity",
    # "u_component_of_wind",
    # "v_component_of_wind"
]
in_vars = [f"{dataset}:{var}" for var in variables]
out_vars = [f"{dataset}:{var}" for var in variables]

train_years = range(1979, 2015)
val_years = range(2015, 2017)
test_years = range(2017, 2019)
history = 3
subsample = 6
pred_range = 6  # CHANGE ME

In [3]:
forecasting_args = ForecastingArgs(
    in_vars,
    out_vars,
    history=history,
    pred_range=pred_range,
    subsample=subsample
)

train_dataset_args = ShardDatasetArgs(
    ERA5Args(root, variables, train_years, name=dataset),
    forecasting_args,
    n_chunks=4
)
val_dataset_args = MapDatasetArgs(
    ERA5Args(root, variables, val_years, name=dataset),
    forecasting_args
)
test_dataset_args = MapDatasetArgs(
    ERA5Args(root, variables, test_years, name=dataset),
    forecasting_args
)

dm = DataModule(
    train_dataset_args,
    val_dataset_args,
    test_dataset_args,
    batch_size=32,
    num_workers=0
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 38.55it/s]


In [4]:
climatology = cl.load_forecasting_module(data_module=dm, preset="climatology")
persistence = cl.load_forecasting_module(data_module=dm, preset="persistence")
linreg = cl.load_forecasting_module(data_module=dm, preset="linear-regression")

Loading preset: climatology


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 38.08it/s]


Using preset optimizer
Using preset learning rate scheduler
Loading training loss: lat_mse
Loading validation loss: lat_rmse
Loading validation loss: lat_acc
Loading validation loss: lat_rmse
Loading validation loss: lat_acc
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading preset: persistence


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 37.67it/s]


Using preset optimizer
Using preset learning rate scheduler
Loading training loss: lat_mse
Loading validation loss: lat_rmse
Loading validation loss: lat_acc
Loading validation loss: lat_rmse
Loading validation loss: lat_acc
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading preset: linear-regression


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 37.44it/s]


Using preset optimizer
Using preset learning rate scheduler
Loading training loss: lat_mse
Loading validation loss: lat_rmse
Loading validation loss: lat_acc
Loading validation loss: lat_rmse
Loading validation loss: lat_acc
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading validation transform: denormalize
Loading validation transform: denormalize


In [5]:
trainer = cl.Trainer(
    early_stopping="lat_rmse:aggregate",
    patience=5,
    accelerator="gpu",
    devices=[0]
)

Global seed set to 0
/home/jason.jewik/climate-learn/src/climate_learn/trainer.py:48: UserWarning: In interactive environment: cannot use DDP spawn strategy
  warn("In interactive environment: cannot use DDP spawn strategy")


In [ ]:
trainer.fit(linreg, dm)

/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/pytorch_lightning/loops/utilities.py:97: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,


┏━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name       ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ net        │ LinearRegression │ 12.6 M │
│ 1 │ net.linear │ Linear           │ 12.6 M │
└───┴────────────┴──────────────────┴────────┘

Trainable params: 12.6 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 12.6 M                                                                                               
Total estimated model params size (MB): 50

Output()

/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_con
nector.py:229: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on 
this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,

/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_con
nector.py:229: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a 
bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on 
this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,

In [6]:
trainer.test(climatology, dm)

Output()

/home/jason.jewik/miniconda3/envs/climate/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:229: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric          ┃         DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      lat_acc:aggregate       │     0.016794127112900886     │
│ lat_acc:era5:2m_temperature  │     0.016794127112900886     │
│      lat_rmse:aggregate      │      6103.093275199047       │
│ lat_rmse:era5:2m_temperature │      6103.093275199047       │
└──────────────────────────────┴──────────────────────────────┘

In [7]:
trainer.test(persistence, dm)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric          ┃         DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      lat_acc:aggregate       │      0.8519486785048739      │
│ lat_acc:era5:2m_temperature  │      0.8519486785048739      │
│      lat_rmse:aggregate      │      2.7992303566951207      │
│ lat_rmse:era5:2m_temperature │      2.7992303566951207      │
└──────────────────────────────┴──────────────────────────────┘

In [8]:
trainer.test(linreg, dm)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric          ┃         DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      lat_acc:aggregate       │    -0.011611564808748646     │
│ lat_acc:era5:2m_temperature  │    -0.011611564808748646     │
│      lat_rmse:aggregate      │      21.367626966645556      │
│ lat_rmse:era5:2m_temperature │      21.367626966645556      │
└──────────────────────────────┴──────────────────────────────┘